In [1]:
%load_ext autoreload
%autoreload 2

# LSTM Experiment

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch_geometric.data import Batch

In [3]:
train_npz = np.load('../../train.npz')
train_data = train_npz['data']
test_npz  = np.load('../../test_input.npz')
test_data  = test_npz['data']

In [4]:
print(train_data.shape, test_data.shape)

# Split once for later use
X_train = train_data[..., :50, :]
Y_train = train_data[:, 0, 50:, :2]

(10000, 50, 110, 6) (2100, 50, 50, 6)


In [5]:
import sys
sys.path.append('..')
from TrajectoryDataset import TrajectoryDatasetTrain, TrajectoryDatasetTest
from utils import train_model

sys.path.append('../..')
from models.lstm import LSTMModel

In [6]:
torch.manual_seed(251)
np.random.seed(42)

scale = 5.0

N = len(train_data)
val_size = int(0.05 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using Apple Silicon GPU


In [7]:
model = LSTMModel()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25) # You can try different schedulers
early_stopping_patience = 10
criterion = nn.MSELoss()

In [8]:
model = model.to(device)
train_model(
    model, 
    train_dataloader, 
    val_dataloader, 
    device, 
    optimizer, 
    criterion, 
    scheduler, 
    early_stopping_patience,
    "lstm_weights"
)

Epoch:   1%|          | 1/100 [00:05<08:55,  5.41s/epoch]

Epoch 000 | Learning rate 0.001000 | train normalized MSE   4.0937 | val normalized MSE   1.4773, | val unnormalized MAE   3.4265 | val unnormalized MSE  36.9317


Epoch:   2%|▏         | 2/100 [00:09<07:49,  4.79s/epoch]

Epoch 001 | Learning rate 0.001000 | train normalized MSE   1.0004 | val normalized MSE   0.8170, | val unnormalized MAE   2.3682 | val unnormalized MSE  20.4262


Epoch:   3%|▎         | 3/100 [00:13<07:18,  4.52s/epoch]

...

Epoch 055 | Learning rate 0.000063 | train normalized MSE   0.3452 | val normalized MSE   0.3814, | val unnormalized MAE   1.4858 | val unnormalized MSE   9.5355


Epoch:  57%|█████▋    | 57/100 [04:12<03:20,  4.66s/epoch]

Epoch 056 | Learning rate 0.000063 | train normalized MSE   0.3485 | val normalized MSE   0.3805, | val unnormalized MAE   1.4799 | val unnormalized MSE   9.5113


Epoch:  57%|█████▋    | 57/100 [04:16<03:13,  4.51s/epoch]

Epoch 057 | Learning rate 0.000063 | train normalized MSE   0.3451 | val normalized MSE   0.3848, | val unnormalized MAE   1.4794 | val unnormalized MSE   9.6211
Early stop!


In [9]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

lstm_weights = torch.load("lstm_weights.pt")
model.load_state_dict(lstm_weights)

# also save entire model
torch.save(model, "full_lstm_model.pt")

test_dataset.generate_submission_predictions(model, device, output_file_name='lstm_submission')